In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

dosya_yolu = 'turkish_tv_series.xlsx' 
df = pd.read_excel(dosya_yolu)


In [2]:
# yeterli bilgisi olmayan satirların silinmesi
df = df[df['Dizi İsimleri'] != 0]
df = df.reset_index(drop=True)

# gosterim suresinin int e cevirilmesi ve boş degerlerin platform ortalamasi ile doldurulmasi
df['Gösterim Süresi'] = df['Gösterim Süresi'].replace(0, '0 dakika')
df['Gösterim Süresi'] = df['Gösterim Süresi'].str.replace('dakika','')
df['Gösterim Süresi'] = df['Gösterim Süresi'].str.replace('Dakika','')
df['Gösterim Süresi'] = df['Gösterim Süresi'].apply(lambda  x: x.strip()).astype(int)
platform_ortalamalari = df.groupby('Platform')['Gösterim Süresi'].mean().apply(np.ceil).astype(int).to_dict()
for index, row in df.iterrows():
    if row['Gösterim Süresi'] == 0:
        platform = row['Platform']
        ortalama = platform_ortalamalari[platform]
        df.at[index, 'Gösterim Süresi'] = ortalama

In [ ]:
# basrol oyuncularinin dizilerdeki ortalma sezon sayisi dataframe e ekleniyor
veriler = {}
with open('actor_season_average.txt', 'r') as dosya:
    for satir in dosya:
        oyuncu, ortalama = satir.strip().split(':')
        veriler[oyuncu] = float(ortalama)

def sezon_ortalama_hesapla(basroller):
    oyuncular = basroller.split(',')
    toplam_ortalama = sum(veriler.get(oyuncu, 0) for oyuncu in oyuncular)
    return toplam_ortalama / len(oyuncular)

df['Başrollerin Sezon Ortalaması'] = df['Başroller'].apply(sezon_ortalama_hesapla)

In [ ]:
# yonetmenlerin yonettikleri dizilerdeki ortalma sezon sayisi dataframe e ekleniyor
veriler = {}
with open('director_season_average.txt', 'r') as dosya:
    for satir in dosya:
        oyuncu, ortalama = satir.strip().split(':')
        veriler[oyuncu] = float(ortalama)

def sezon_ortalama_hesapla(basroller):
    oyuncular = basroller.split(',')
    toplam_ortalama = sum(veriler.get(oyuncu, 0) for oyuncu in oyuncular)
    return toplam_ortalama / len(oyuncular)

df['Yönetmen Sezon Ortalaması'] = df['Yönetmen'].apply(sezon_ortalama_hesapla)

In [ ]:
# senaristlerin senaryosunu yazdigi dizilerdeki ortalma sezon sayisi dataframe e ekleniyor
veriler = {}
with open('scriptwriter_season_average.txt', 'r') as dosya:
    for satir in dosya:
        oyuncu, ortalama = satir.strip().split(':')
        veriler[oyuncu] = float(ortalama)

def sezon_ortalama_hesapla(basroller):
    oyuncular = basroller.split(',')
    toplam_ortalama = sum(veriler.get(oyuncu, 0) for oyuncu in oyuncular)
    return toplam_ortalama / len(oyuncular)

df['Senarist Sezon Ortalaması'] = df['Senarist'].apply(sezon_ortalama_hesapla)

In [3]:
with open('award_winning_directors.txt', 'r') as dosya:
    yonetmenler = dosya.readlines()

yonetmenler = [yonetmen.strip() for yonetmen in yonetmenler]

df['Ödüllü Yönetmen'] = df['Yönetmen'].apply(lambda x: 1 if x in yonetmenler else 0)


with open('award_winning_actors.txt', 'r') as dosya2:
    oyuncular = dosya2.readlines()

oyuncular = [oyuncu.strip() for oyuncu in oyuncular]

df['Ödüllü Aktör'] = df['Başroller'].apply(lambda x: 1 if any(oyunc in oyuncular for oyunc in x.split(',')) else 0)


with open('exported_series.txt', 'r') as dosya3:
    diziler = dosya3.readlines()

diziler = [dizi.strip() for dizi in diziler]

df['İhraç'] = df['Dizi İsimleri'].apply(lambda x: 1 if any(diz in diziler for diz in x.split(',')) else 0)


with open('award_winning_series.txt', 'r') as dosya4:
    diziler = dosya4.readlines()

diziler = [dizi.strip() for dizi in diziler]

df['Ödüllü Dizi'] = df['Dizi İsimleri'].apply(lambda x: 1 if any(diz in diziler for diz in x.split(',')) else 0)

In [4]:
for index, row in df.iterrows():
    if row['Sezon Sayısı'] == 1:
        df.loc[index, 'Sonuç'] = 'Başarısız'
    elif row['Sezon Sayısı'] == 2:
        df.loc[index, 'Sonuç'] = 'Kısmen Başarılı'
    elif row['Sezon Sayısı'] < 5:
        df.loc[index, 'Sonuç'] = 'Başarılı'
    elif row['Sezon Sayısı'] < 10:
        df.loc[index, 'Sonuç'] = 'Kısmen Olgun'
    elif row['Sezon Sayısı'] >= 10:
        df.loc[index, 'Sonuç'] = 'Olgun'

In [ ]:
df.drop('Türler', axis=1, inplace=True)
df.drop('Sezon Sayısı', axis=1, inplace=True)
df.drop('Bölüm Sayısı', axis=1, inplace=True)

In [7]:
dosya_yolu = 'tv_series_dataset.csv'
df.to_csv(dosya_yolu)